In [1]:
import numpy as np
import inspect

In [2]:
N=10

In [3]:
diag_CT = 1e9*np.array([1, 1, 1 ])  
CT=np.diag(diag_CT)

In [4]:
diag_CR = 1e11*np.array([1, 1, 1])
CR=np.diag(diag_CR)

In [5]:
φ = np.random.rand(N)
θ = np.random.rand(N)
ψ = np.random.rand(N)

In [6]:
φ

array([0.48940863, 0.10329455, 0.68896141, 0.56284692, 0.04122142,
       0.64886993, 0.3755373 , 0.06979395, 0.80965999, 0.50523568])

In [7]:
dφ = np.random.rand(N)
dθ = np.random.rand(N)
dψ = np.random.rand(N)

In [8]:
ddφ = np.random.rand(N)
ddθ = np.random.rand(N)
ddψ = np.random.rand(N)

In [9]:
x = np.random.rand(N)
y = np.random.rand(N)
z = np.random.rand(N)

In [10]:
def Π0(φθψ):
    φ0,θ0,ψ0=φθψ
    return np.array([[np.cos(θ0),0,np.cos(φ0)*np.sin(θ0)],
                  [0,1,-np.sin(φ0)],
                  [-np.sin(θ0),0,np.cos(φ0)*np.cos(θ0)]])

In [11]:
def Π(φ,θ,ψ):
    φθψ=np.stack((φ,θ,ψ), axis=1)
    return np.apply_along_axis(Π0, 1, φθψ)

In [12]:
Π(φ,θ,ψ)

array([[[ 0.94043034,  0.        ,  0.30007576],
        [ 0.        ,  1.        , -0.47010402],
        [-0.33998642,  0.        ,  0.83003418]],

       [[ 0.98780701,  0.        ,  0.15485355],
        [ 0.        ,  1.        , -0.10311096],
        [-0.15568336,  0.        ,  0.98254186]],

       [[ 0.97716072,  0.        ,  0.16403142],
        [ 0.        ,  1.        , -0.63573583],
        [-0.21250161,  0.        ,  0.7542769 ]],

       [[ 0.99981358,  0.        ,  0.01632981],
        [ 0.        ,  1.        , -0.53359611],
        [-0.01930832,  0.        ,  0.84558177]],

       [[ 0.89179458,  0.        ,  0.45205618],
        [ 0.        ,  1.        , -0.04120974],
        [-0.45244052,  0.        ,  0.89103702]],

       [[ 0.71585196,  0.        ,  0.55634435],
        [ 0.        ,  1.        , -0.60428639],
        [-0.69825208,  0.        ,  0.57036736]],

       [[ 0.71223884,  0.        ,  0.65301968],
        [ 0.        ,  1.        , -0.36677243],
        

In [13]:
def w_e(φ,θ,ψ,dφ,dθ,dψ): 
    return np.einsum('ijk,ik->ij', Π(φ,θ,ψ), np.stack((dφ,dθ,dψ), axis=1))

In [14]:
w_e(φ,θ,ψ,dφ,dθ,dψ)

array([[ 1.01643468, -0.17371049,  0.46803597],
       [ 0.59289055,  0.23805279,  0.41135039],
       [ 0.81289289,  0.49086338,  0.28351642],
       [ 0.29352065,  0.39104278,  0.52059679],
       [ 0.50966246,  0.81281579, -0.08099943],
       [ 0.78907744, -0.2596705 , -0.06132528],
       [ 0.91422746,  0.0782029 , -0.40891347],
       [ 0.89857402,  0.9162862 ,  0.61977694],
       [ 0.73593537,  0.16107834, -0.40176111],
       [ 0.66454905,  0.06252698,  0.22616086]])

In [15]:
def dw_e(φ,θ,ψ,dφ,dθ,dψ,ddφ,ddθ,ddψ):
    return (np.einsum('ijk,ik->ij', Π(φ,θ,ψ), np.stack((ddφ,ddθ,ddψ), axis=1))+
        np.einsum('ijk,ik->ij', Π(dφ,dθ,dψ), np.stack((dφ,dθ,dψ), axis=1)))

In [16]:
dw_e(φ,θ,ψ,dφ,dθ,dψ,ddφ,ddθ,ddψ)

array([[ 1.3747771 , -0.51558639,  0.70906516],
       [ 1.21326797,  0.31762484,  0.61473473],
       [ 1.60891809,  0.73486123, -0.19592448],
       [ 1.54492382,  1.26109378,  0.43676192],
       [ 1.02960115,  0.91381577, -0.22086948],
       [ 1.11149192,  0.22097777,  0.02638711],
       [ 1.90590505,  0.58476011,  0.00266687],
       [ 0.92868139,  0.69067285, -0.0468101 ],
       [ 1.85621954,  0.35566286, -0.0518067 ],
       [ 1.44498022,  0.69973328,  0.1945262 ]])

In [17]:
def phi(x,y,z): return np.stack((x,y,z), axis=1) 
def theta(φ,θ,ψ): return np.stack((φ,θ,ψ), axis=1) 

In [18]:
phi(x,y,z)

array([[0.40864751, 0.57765141, 0.14428315],
       [0.78883778, 0.3630431 , 0.75689744],
       [0.10536405, 0.76829776, 0.55214602],
       [0.2138908 , 0.47244844, 0.50279996],
       [0.11647896, 0.65663744, 0.84385902],
       [0.82205414, 0.86071075, 0.74162797],
       [0.64632682, 0.31147873, 0.4474258 ],
       [0.46003092, 0.91138156, 0.51954872],
       [0.61355093, 0.65032005, 0.68580474],
       [0.06216346, 0.02621091, 0.21559772]])

In [19]:
theta(φ,θ,ψ)

array([[0.48940863, 0.34690246, 0.93556631],
       [0.10329455, 0.15631921, 0.02516144],
       [0.68896141, 0.21413434, 0.39860853],
       [0.56284692, 0.01930952, 0.98486305],
       [0.04122142, 0.46950009, 0.38011434],
       [0.64886993, 0.77295285, 0.76873788],
       [0.3755373 , 0.77811374, 0.37204742],
       [0.06979395, 0.01282654, 0.23442888],
       [0.80965999, 0.64113818, 0.18763575],
       [0.50523568, 0.22733868, 0.72653049]])

In [20]:
def ω(φ,θ,ψ): 
    return np.einsum('ijk,ik->ij', Π(φ,θ,ψ), theta(φ,θ,ψ))

In [21]:
ω(x,y,z)

array([[ 0.41464297,  0.52031782, -0.11222515],
       [ 0.92683   , -0.17400194,  0.21846632],
       [ 0.45733188,  0.710229  ,  0.3216246 ],
       [ 0.41405464,  0.36572229,  0.3401843 ],
       [ 0.60390614,  0.55856772,  0.5927437 ],
       [ 0.91871881,  0.31743439, -0.29426451],
       [ 0.72469042,  0.04201289,  0.14191606],
       [ 0.64977662,  0.68071449, -0.07837329],
       [ 0.82780198,  0.25545127,  0.0748024 ],
       [ 0.06778156,  0.01281724,  0.2134782 ]])

In [22]:
def Ret(φ,θ,ψ):
    
    def Ret0(arr):
        φ,θ,ψ=arr
        Cφ=np.matrix([[1,0,0],
                      [0,np.cos(φ),-np.sin(φ)],
                      [0,np.sin(φ),np.cos(φ)]])

        Cθ=np.matrix([[np.cos(θ),0,np.sin(θ)],
                      [0,1,0],
                      [-np.sin(θ),0,np.cos(θ)]])

        Cψ=np.matrix([[np.cos(ψ),-np.sin(ψ),0],
                      [np.sin(ψ),np.cos(ψ),0],
                      [0,0,1]])


        return np.array(np.dot(np.dot(Cθ,Cφ),Cψ) )
    
    arr=np.stack((φ,θ,ψ), axis=1)
    return np.apply_along_axis(Ret0, 1, arr)

In [23]:
Ret(φ,θ,ψ)

array([[[ 0.68666825, -0.66214896,  0.30007576],
        [ 0.71044471,  0.52370844, -0.47010402],
        [ 0.15412668,  0.53599274,  0.83003418]],

       [[ 0.9878982 , -0.00880445,  0.15485355],
        [ 0.02502469,  0.99435501, -0.10311096],
        [-0.15307156,  0.1057383 ,  0.98254186]],

       [[ 0.95298847, -0.25476786,  0.16403142],
        [ 0.29960504,  0.71139073, -0.63573583],
        [ 0.04527463,  0.65499356,  0.7542769 ]],

       [[ 0.56145844, -0.8273438 ,  0.01632981],
        [ 0.70466702,  0.46767465, -0.53359611],
        [ 0.43383039,  0.31109911,  0.84558177]],

       [[ 0.83505804, -0.31356544,  0.45205618],
        [ 0.37071147,  0.92783337, -0.04120974],
        [-0.40651086,  0.20199494,  0.89103702]],

       [[ 0.807894  , -0.194392  ,  0.55634435],
        [ 0.55393533,  0.57270726, -0.60428639],
        [-0.20115401,  0.79637814,  0.57036736]],

       [[ 0.75710061, -0.01907789,  0.65301968],
        [ 0.33818984,  0.8666635 , -0.36677243],
        

In [24]:
def d_s(func, a, b, c, φ, θ, ψ): 
    
    sig = inspect.signature(func)
    num_args = len(sig.parameters)
    
    if num_args==3:
        fu = func(a, b, c)
    else:
        fu = func(a, b, c, φ, θ, ψ)

    def S(arr):
        a1, a2, a3 = arr[0],arr[1],arr[2]
        return np.array([[0, -a3, a2 ],
                         [a3, 0, -a1],
                        [-a2, a1, 0]])
    
    S0 = np.apply_along_axis(S, 1, ω(φ,θ,ψ))
    
    return np.einsum('ijk,ik->ij',S0, fu).T

In [25]:
d_s(phi,x,y,z,φ, θ, ψ)

array([[-0.36586512,  0.11321687, -0.14020013, -0.64283443,  0.17281933,
         0.24130573,  0.29241511, -0.2141304 ,  0.59399902, -0.04005171],
       [ 0.14242709, -0.07336293, -0.39156775, -0.11523302, -0.1387454 ,
        -0.67367569, -0.23941966,  0.06934454, -0.73157072, -0.10559274],
       [ 0.46600495, -0.08280625,  0.57161089,  0.38173836,  0.08410837,
         0.51437317, -0.25573312,  0.06795756,  0.16230051,  0.02438539]])

In [26]:
d_s(theta, φ, θ, ψ, φ, θ, ψ)

array([[-2.98590336e-01,  2.51719456e-03, -4.86869673e-02,
        -5.14417874e-01,  2.22479595e-02,  2.48384529e-01,
         2.52019901e-01, -3.81349599e-03,  3.38790079e-01,
        -2.05242324e-01],
       [-3.94632989e-01, -1.77282674e-03, -1.88140096e-01,
        -1.07450870e-01, -6.60968621e-02, -6.95332012e-01,
        -1.96318448e-01, -8.05481191e-04, -4.44387907e-01,
        -2.06319045e-01],
       [ 3.02524620e-01,  6.80182926e-04,  1.85221065e-01,
         2.96095322e-01,  7.92271873e-02,  4.89490136e-01,
         1.56203769e-01,  1.17942156e-03,  5.65418984e-02,
         2.07286612e-01]])

In [27]:
def ne(x, y, z, φ, θ, ψ):
    RET = Ret(φ,θ,ψ)
    A = np.einsum('ijk,ikp->ijp',np.einsum('ijk,kp->ijp',RET, CT), np.transpose(RET,(0,2,1))) 
    B = d_s(phi, x, y, z, φ, θ, ψ).T
    t0 = np.diff(phi(x,y,z),axis=0)
    t1 = np.insert(t0, 0, [0,0,0], axis=0)
    return np.einsum('ijk,ik->ij', A, B - t1 )

In [28]:
ne(x, y, z, φ, θ, ψ)

array([[-3.65865121e+08,  1.42427086e+08,  4.66004954e+08],
       [-2.66973394e+08,  1.41245370e+08, -6.95420534e+08],
       [ 5.43273601e+08, -7.96822402e+08,  7.76362309e+08],
       [-7.51361182e+08,  1.80616295e+08,  4.31084415e+08],
       [ 2.70231168e+08, -3.22934393e+08, -2.56950686e+08],
       [-4.64269449e+08, -8.77749004e+08,  6.16604216e+08],
       [ 4.68142429e+08,  3.09812355e+08,  3.84690532e+07],
       [-2.78344961e+07, -5.30558279e+08, -4.16535781e+06],
       [ 4.40479015e+08, -4.70509213e+08, -3.95550312e+06],
       [ 5.11335752e+08,  5.18516397e+08,  4.94592412e+08]])

In [29]:
d_s(ne, x, y, z, φ, θ, ψ)

array([[-1.30200271e+08, -1.08123867e+08,  9.24225386e+07,
        -3.66670226e+08, -1.32596205e+07,  1.77344514e+08,
         2.99766805e+07,  1.23604882e+08, -1.81048240e+08,
        -3.23607837e+08],
       [-5.68543737e+08,  7.13599896e+07, -4.89626138e+08,
        -8.67077003e+08,  1.40084717e+08, -5.43336778e+08,
        -2.76332334e+07, -6.18068383e+06, -1.64748973e+08,
        -5.58325124e+07],
       [ 7.15449267e+07,  5.60027522e+07, -5.67203991e+08,
        -2.75801061e+08, -1.90002745e+08, -6.39920496e+08,
        -1.42250419e+08, -3.87161545e+07, -5.64287461e+08,
         3.93096063e+08]])

In [30]:
def me(a, b, c, φ, θ, ψ):
    RET = Ret(φ,θ,ψ)
    A = np.einsum('ijk,ikp->ijp',np.einsum('ijk,kp->ijp', RET, CR), np.transpose(RET,(0,2,1))) 
    return np.einsum('ijk,ik->ij', A, ω(φ,θ,ψ))

In [31]:
me(φ, θ, ψ, φ, θ, ψ)

array([[ 7.40995502e+10, -9.29110259e+09,  6.10159727e+10],
       [ 1.05931425e+10,  1.53724790e+10,  8.64092948e+08],
       [ 7.38610351e+10, -3.92753905e+09,  1.54255793e+10],
       [ 5.78824617e+10, -5.06209568e+10,  8.21914608e+10],
       [ 2.08594074e+10,  4.53835674e+10,  3.20045711e+10],
       [ 8.92177788e+10,  3.08415015e+10, -1.46117908e+09],
       [ 5.10426539e+10,  6.41657000e+10, -1.70837062e+09],
       [ 7.27877139e+09, -3.52189939e+08,  2.32943707e+10],
       [ 7.26281673e+10,  5.05280014e+10, -3.80544309e+10],
       [ 6.35526441e+10, -1.24312025e+10,  5.05527028e+10]])

In [32]:
d_s(me, φ, θ, ψ, φ, θ, ψ)

array([[ 9.53674316e-07,  2.98023224e-08,  0.00000000e+00,
         7.62939453e-06, -1.90734863e-06,  1.19209290e-06,
         4.76837158e-07, -1.49011612e-07, -7.62939453e-06,
        -1.90734863e-06],
       [ 1.32728237e-06, -7.22877101e-09, -4.78430401e-07,
         1.72854760e-05,  1.47323194e-06, -3.33828228e-06,
        -3.41602950e-07,  4.68975404e-07,  6.28625226e-06,
        -9.71789717e-06],
       [-9.53674316e-07, -2.38418579e-07, -4.76837158e-07,
         3.81469727e-06,  0.00000000e+00,  0.00000000e+00,
         3.81469727e-06,  5.58793545e-08,  0.00000000e+00,
         0.00000000e+00]])